In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df=pd.read_csv("../content/drive/MyDrive/Twitter_Data.csv")

In [4]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [5]:
import numpy as np

In [6]:
num_classes=len(df["category"].value_counts())

In [7]:
df['Labels']=df['category'].map({0.0:0,
                                 1.0:1,
                                 -1.0:2})

In [8]:
df.head()

,clean_text,category,Labels
0,when modi promised “minimum government maximum...,-1.0,2.0
1,talk all the nonsense and continue all the dra...,0.0,0.0
2,what did just say vote for modi welcome bjp t...,1.0,1.0
3,asking his supporters prefix chowkidar their n...,1.0,1.0
4,answer who among these the most powerful world...,1.0,1.0


In [9]:
df.drop(['category'],axis=1,inplace=True)

In [10]:
df.head()

,clean_text,Labels
0,when modi promised “minimum government maximum...,2.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [11]:
df['Labels'].values

array([2., 0., 1., ..., 0., 0., 1.])

In [12]:
df.isnull().sum()

clean_text    4
Labels        7
dtype: int64

In [13]:
df.dropna(inplace=True)

In [14]:
df.isnull().sum()

clean_text    0
Labels        0
dtype: int64

In [15]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

y=tf.keras.utils.to_categorical(df['Labels'].values,num_classes=num_classes)
x_train,x_test,y_train,y_test=train_test_split(df['clean_text'],y,test_size=0.2)

In [16]:
y_train

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [18]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 6.3 MB/s 
     |████████████████████████████████| 578.0 MB 17 kB/s 
     |████████████████████████████████| 5.9 MB 57.2 MB/s 
     |████████████████████████████████| 1.7 MB 48.0 MB/s 
     |████████████████████████████████| 438 kB 65.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [17]:
import tensorflow_hub as hub
import tensorflow_text as text

encoder=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
preprocessor=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [18]:
def get_embeddings(sentences):
  preprocessed_text=preprocessor(sentences)
  return encoder(preprocessed_text)['pooled_output']

In [19]:
from keras import backend as K

def balanced_recall(y_true, y_pred):
    """This function calculates the balanced recall metric
    recall = TP / (TP + FN)
    """
    recall_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true_class, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        recall_by_class = recall_by_class + recall
    return recall_by_class / y_pred.shape[1]

def balanced_precision(y_true, y_pred):
    """This function calculates the balanced precision metric
    precision = TP / (TP + FP)
    """
    precision_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred_class, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        precision_by_class = precision_by_class + precision
    # return average balanced metric for each class
    return precision_by_class / y_pred.shape[1]

def balanced_f1_score(y_true, y_pred):
    """This function calculates the F1 score metric"""
    precision = balanced_precision(y_true, y_pred)
    recall = balanced_recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [20]:
i=tf.keras.layers.Input(shape=(),dtype=tf.string,name='text')
x=preprocessor(i)
x=encoder(x)
x=tf.keras.layers.Dropout(0.2,name='dropout')(x['pooled_output'])
x=tf.keras.layers.Dense(num_classes,activation='softmax',name="output")(x)

In [21]:
model=tf.keras.Model(i,x)

In [22]:
n_epochs =4 

METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
      balanced_recall,
      balanced_precision,
      balanced_f1_score
]

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                      patience = 3,
                                                      restore_best_weights = True)

model.compile(optimizer = "adam",
              loss = "categorical_crossentropy",
              metrics = METRICS)

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [24]:
model_fit=model.fit(x_train,
                    y_train,
                    n_epochs,
                    validation_data=(x_test,y_test),
                    callbacks=[earlystop_callback])

32594/32594 [==============================] - 1734s 53ms/step - loss: 1.0256 - accuracy: 0.5028 - balanced_recall: 0.2778 - balanced_precision: 0.2882 - balanced_f1_score: 0.2678 - val_loss: 0.9982 - val_accuracy: 0.5135 - val_balanced_recall: 0.3243 - val_balanced_precision: 0.2493 - val_balanced_f1_score: 0.2669
